# Preparar para classificação

In [1]:
import numpy as np
import pandas as pd

In [2]:
# declare a list tasks whose products you want to use as inputs
upstream = ['agrupar_dados']
product = None
not_classification_columns = []
feature_selection_percentage = 0.8 # colunas que vão restar após a selecao de feature
dimensionality_reduction_percentage = 0.1 # colunas que vão restar após a redução de dimensionalidade
dimensionality_reduction_algoritm = 'svd' # pca ou svd

In [3]:
# Parameters
not_classification_columns = [
    "business_id",
    "attributes",
    "categories",
    "hours",
    "destaque",
    "graph_page_rank",
]
feature_selection_percentage = 0.8
dimensionality_reduction_percentage = 0.1
dimensionality_reduction_algoritm = "svd"
upstream = {
    "agrupar_dados": {
        "data": "/home/guilherme/\xc1rea de Trabalho/Estudo Faculdade/Disciplinas/minera\xe7\xe3o de dados/Desafio/pipelines/processamento_features/output/agrupar_dados_data.parquet",
        "nb": "/home/guilherme/\xc1rea de Trabalho/Estudo Faculdade/Disciplinas/minera\xe7\xe3o de dados/Desafio/pipelines/processamento_features/output/output_agrupar_dados.ipynb",
    }
}
product = {
    "data_X": "/home/guilherme/\xc1rea de Trabalho/Estudo Faculdade/Disciplinas/minera\xe7\xe3o de dados/Desafio/pipelines/processamento_features/output/preparar_classificacao_data_X.parquet",
    "data_y": "/home/guilherme/\xc1rea de Trabalho/Estudo Faculdade/Disciplinas/minera\xe7\xe3o de dados/Desafio/pipelines/processamento_features/output/preparar_classificacao_data_y.parquet",
    "nb": "/home/guilherme/\xc1rea de Trabalho/Estudo Faculdade/Disciplinas/minera\xe7\xe3o de dados/Desafio/pipelines/processamento_features/output/output_preparar_classificacao.ipynb",
}


In [4]:
df_reg = pd.read_parquet(upstream['agrupar_dados']['data'])
df_reg

,business_id,destaque,latitude,longitude,review_count,business_open_Friday_afternoon,business_open_Friday_evening,business_open_Friday_morning,business_open_Friday_night,business_open_Monday_afternoon,...,attribute_Smoking_outdoor,attribute_Smoking_yes,attribute_WheelchairAccessible_False,attribute_WheelchairAccessible_True,attribute_WiFi_free,attribute_WiFi_no,attribute_WiFi_paid,attributes,score,stars
0,vHzWmPWHN4J1hRR3W3AMQg,0,-0.686427,-0.425989,1.190565,0.000000,1.000000,0.000000,0.166667,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,"{'AcceptsInsurance': None, 'AgesAllowed': None...",0.532271,-0.080982
1,15to24Q-otAHmto7FzsWRg,0,-0.569901,-0.956974,-0.400175,0.833333,0.000000,0.500000,0.000000,0.833333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'AcceptsInsurance': None, 'AgesAllowed': None...",0.577041,0.841286
2,8aqKdf4G4AAir8k_Kdslvg,0,-0.697652,0.198579,-0.151622,1.000000,0.166667,0.833333,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'AcceptsInsurance': True, 'AgesAllowed': None...",0.706820,-1.153910
3,uxU1vr5AhhkTQ83X0bpeyg,0,2.177144,-0.889070,-0.400175,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'AcceptsInsurance': None, 'AgesAllowed': None...",0.400056,0.288943
4,f702hTJoqdR34Jn23C7d1A,0,-0.312937,-1.101228,-0.400175,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'AcceptsInsurance': None, 'AgesAllowed': None...",0.511156,-2.627026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17577,Zp5xil8pC6Bm-fFOWIbh0Q,0,2.264886,2.676289,-0.184762,1.000000,0.833333,0.166667,0.000000,1.000000,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,"{'AcceptsInsurance': None, 'AgesAllowed': None...",0.567692,-1.241298
17578,o2811xZ0ufRi7WcsqWiGzw,0,-0.196596,0.115073,-0.383605,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'AcceptsInsurance': None, 'AgesAllowed': None...",0.757226,0.757598
17579,v9T9m_TbpLuaC2Kud5UbLA,0,0.029843,0.907477,-0.317324,1.000000,0.000000,0.333333,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'AcceptsInsurance': None, 'AgesAllowed': None...",0.657578,-0.147039
17580,bzGnAIe9OiBHGWJE-dZ_8g,0,1.553877,2.925831,-0.300754,1.000000,1.000000,0.083333,0.333333,1.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,"{'AcceptsInsurance': None, 'AgesAllowed': None...",0.696177,-2.010238


## Seleção de features

In [5]:
classification_columns = df_reg.columns.difference(not_classification_columns)
df_X = df_reg[classification_columns]
df_y = df_reg['destaque']

In [6]:
category_columns = [col for col in df_reg.columns if col.startswith( 'category_')]
attributes_columns = [col for col in df_reg.columns if col.startswith( 'attribute_')]
hours_columns = [col for col in df_reg.columns if col.startswith( 'business_open_')]
geo_columns = ['latitude', 'longitude']
print("category lenght:", len(category_columns))
print("attributes lenght:", len(attributes_columns))
print("hours lenght:", len(hours_columns))
print("geo lenght:", len(geo_columns))

category lenght: 897
attributes lenght: 150
hours lenght: 28
geo lenght: 2


In [7]:
from sklearn.feature_selection import SelectKBest, f_classif
import math

def select_best(columns):
    features_num = math.ceil(feature_selection_percentage * len(columns))
    selector = SelectKBest(f_classif, k=features_num)
    X_new = pd.DataFrame(selector.fit_transform(df_X[columns], df_y), 
                        columns=[list(df_X[columns].iloc[:, selector.get_support(indices=True)].columns)])
    X_new.columns = [''.join(col) for col in X_new.columns.values]
    return X_new

df_X_best_cat = select_best(category_columns)
df_X_best_attr = select_best(attributes_columns)
df_X_best_hours = select_best(hours_columns)
df_X_select = df_X.drop(category_columns + attributes_columns + hours_columns, axis=1)
df_X_select = pd.concat([df_X_select, df_X_best_attr, df_X_best_cat, df_X_best_hours], axis=1)
df_X_select

,graph_betweenness_centrality,graph_closeness_centrality,latitude,longitude,review_count,score,stars,attribute_AcceptsInsurance_False,attribute_AcceptsInsurance_True,attribute_Alcohol_full_bar,...,business_open_Sunday_night,business_open_Thursday_afternoon,business_open_Thursday_morning,business_open_Thursday_night,business_open_Tuesday_afternoon,business_open_Tuesday_morning,business_open_Tuesday_night,business_open_Wednesday_afternoon,business_open_Wednesday_morning,business_open_Wednesday_night
0,1.674021e-04,0.466521,-0.686427,-0.425989,1.190565,0.532271,-0.080982,0.0,0.0,1.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,6.304093e-05,0.451816,-0.569901,-0.956974,-0.400175,0.577041,0.841286,0.0,0.0,0.0,...,0.0,0.833333,0.500000,0.000000,0.833333,0.500000,0.000000,0.833333,0.500000,0.000000
2,1.197574e-05,0.439563,-0.697652,0.198579,-0.151622,0.706820,-1.153910,0.0,1.0,0.0,...,0.0,1.000000,0.833333,0.000000,1.000000,0.833333,0.000000,1.000000,0.833333,0.000000
3,3.420556e-05,0.463989,2.177144,-0.889070,-0.400175,0.400056,0.288943,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2.116433e-07,0.388573,-0.312937,-1.101228,-0.400175,0.511156,-2.627026,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17577,8.673320e-07,0.375931,2.264886,2.676289,-0.184762,0.567692,-1.241298,0.0,0.0,0.0,...,0.0,1.000000,0.166667,0.000000,1.000000,0.166667,0.000000,1.000000,0.166667,0.000000
17578,3.307555e-05,0.466878,-0.196596,0.115073,-0.383605,0.757226,0.757598,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
17579,8.215360e-05,0.477130,0.029843,0.907477,-0.317324,0.657578,-0.147039,0.0,0.0,0.0,...,0.0,1.000000,0.333333,0.000000,1.000000,0.333333,0.000000,1.000000,0.333333,0.000000
17580,8.721090e-06,0.365759,1.553877,2.925831,-0.300754,0.696177,-2.010238,0.0,0.0,1.0,...,0.0,1.000000,0.083333,0.166667,1.000000,0.083333,0.166667,1.000000,0.083333,0.166667


## Redução de dimensionalidade

In [8]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

def svd_reduction(columns, features_number):
    svd = TruncatedSVD(n_components=features_number, n_iter=30, random_state=43)
    return svd.fit_transform(df_pca_input[columns])
    
def pca_reduction(columns, features_number):
    svd = PCA(n_components=features_number)
    return svd.fit_transform(df_pca_input[columns])

df_pca_input = df_X_select
if dimensionality_reduction_algoritm == 'pca':
    reduction_algorithm = pca_reduction
elif dimensionality_reduction_algoritm == 'svd':
    reduction_algorithm = svd_reduction
else:
    raise f'algoritmo de redução de dimensionalidade desconhecido: {dimensionality_reduction_algoritm}'

filtered_cat_cols = [col for col in df_pca_input.columns if col in category_columns]
filtered_attr_cols = [col for col in df_pca_input.columns if col in attributes_columns]
filtered_hours_cols = [col for col in df_pca_input.columns if col in hours_columns]

def reduce_dimensions(columns, columns_prefix): 
    features_number = math.ceil(len(columns) * dimensionality_reduction_percentage)
    X_new = pd.DataFrame(reduction_algorithm(columns, features_number), 
                                       columns=[columns_prefix + str(x) for x in range(features_number)])
    return X_new
    
if dimensionality_reduction_percentage < 1.0:
    df_X_pca_cat = reduce_dimensions(filtered_cat_cols, 'cat_pca_')
    df_X_pca_attr = reduce_dimensions(filtered_attr_cols, 'attr_pca_')
    df_X_pca_hours = reduce_dimensions(filtered_hours_cols, 'hours_pca_')
    df_X_pca_geo = reduce_dimensions(geo_columns, 'geolocalization_')
    
    df_X_pca = df_pca_input.drop(filtered_cat_cols + filtered_attr_cols + filtered_hours_cols + geo_columns, axis=1)
    df_X_pca = pd.concat([df_X_pca, df_X_pca_attr, df_X_pca_cat, df_X_pca_hours, df_X_pca_geo], axis=1)
else:
    df_X_pca = df_X

df_X_pca

,graph_betweenness_centrality,graph_closeness_centrality,review_count,score,stars,attr_pca_0,attr_pca_1,attr_pca_2,attr_pca_3,attr_pca_4,...,cat_pca_66,cat_pca_67,cat_pca_68,cat_pca_69,cat_pca_70,cat_pca_71,hours_pca_0,hours_pca_1,hours_pca_2,geolocalization_0
0,1.674021e-04,0.466521,1.190565,0.532271,-0.080982,4.361661,-0.608353,-0.990658,0.689758,-0.149303,...,0.002993,0.009268,0.049751,-0.013704,0.046276,-0.026373,0.371962,0.657449,0.639825,-0.786597
1,6.304093e-05,0.451816,-0.400175,0.577041,0.841286,1.560980,1.958592,-0.497142,0.240193,-0.669506,...,0.018376,0.038694,-0.003010,-0.002021,-0.086792,-0.106295,2.064800,-0.795663,-0.413410,-1.079663
2,1.197574e-05,0.439563,-0.151622,0.706820,-1.153910,0.009820,0.068860,-0.009282,0.023303,-0.018114,...,0.000318,0.574328,-0.034660,-0.032546,0.082137,0.084236,2.733563,-1.114317,-0.229400,-0.352898
3,3.420556e-05,0.463989,-0.400175,0.400056,0.288943,0.016681,0.102161,-0.000553,0.035257,-0.037843,...,0.000318,0.574328,-0.034660,-0.032546,0.082137,0.084236,0.000000,0.000000,0.000000,0.910806
4,2.116433e-07,0.388573,-0.400175,0.511156,-2.627026,0.028192,0.064674,-0.045795,0.006350,0.058997,...,-0.304325,0.064132,-0.019686,-0.071422,0.078996,0.138812,0.000000,0.000000,0.000000,-0.999966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17577,8.673320e-07,0.375931,-0.184762,0.567692,-1.241298,4.380547,-1.348234,1.507152,0.649471,0.365514,...,0.007088,-0.002942,-0.011181,-0.021995,0.049784,-0.011555,2.788913,0.816852,-0.163109,3.493938
17578,3.307555e-05,0.466878,-0.383605,0.757226,0.757598,0.317827,0.080611,-0.027435,-0.035473,-0.014601,...,-0.000715,-0.000890,0.007402,0.006828,-0.014001,0.002576,0.000000,0.000000,0.000000,-0.057645
17579,8.215360e-05,0.477130,-0.317324,0.657578,-0.147039,1.530460,1.924575,0.193756,0.207041,0.451387,...,0.102203,-0.019354,0.005432,0.023825,-0.050687,-0.089870,2.368747,-0.366091,-0.720488,0.662786
17580,8.721090e-06,0.365759,-0.300754,0.696177,-2.010238,3.784400,-0.108334,-0.364065,-1.098206,1.205564,...,0.005830,0.004991,0.060096,-0.045728,0.030437,0.018798,2.824429,1.102188,0.245863,3.167632


In [9]:
df_X_pca.to_parquet(product['data_X'])
pd.DataFrame(df_y).to_parquet(product['data_y'])